In [ ]:
import os
os.chdir("..")

In [ ]:
from src.data.data_process import DataReg
import arviz as az
import bambi as bmb
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os

load_dotenv()

az.style.use("arviz-darkgrid")
dr = DataReg()



In [ ]:
methods = bmb.inference_methods.names
methods

In [ ]:
dr.notify(url=str(os.environ.get("URL")),auth=str(os.environ.get("AUTH")), msg="hi")

In [ ]:
dr.conn.sql("SELECT * FROM qcewtable WHERE ein=='660567825'").pl() #INFO We cound manualy obtain if they are foreing

In [ ]:
data = dr.regular_data(naics="72-food")
data

- rain by naics (2 digits)
- drop null zipcodes
- broke down the 72 (remove food services as its own sector)
- only the first 2 regression(0,1)


In [ ]:
naics_code = [
        "11",
        "21",
        "22",
        "23",
        "31-33",
        "42",
        "44-45",
        "48-49",
        "51",
        "52",
        "54",
        "55",
        "56",
        "61",
        "62",
        "71",
        "72-accommodation",
        "72-food",
        "81",
        "92",
    ]

In [ ]:
import warnings

warnings.filterwarnings(
    "ignore",
    category=FutureWarning
)
warnings.filterwarnings(
    "ignore",
    category=UserWarning
)
for naics in naics_code:
    data = dr.regular_data(naics=naics)
    for i in ["foreign", "local"]:
        if i == "foreign":
            data_pr = data[data["foreign"] == 1]
        else:
            data_pr = data[data["foreign"] == 0]
        print(f"type: {i}, naics:{naics}, length {len(data_pr)}")
    

In [ ]:
data = dr.regular_data(naics=naics)
data_pr = data[data["foreign"] == 0]

In [ ]:
data = dr.regular_data(naics="21")
data_pr = data[data["foreign"] == 0]
model = bmb.Model(
    "log_total_employment ~ 0 + date + ein + log_k_index + own_children6 + own_children17 + commute_car + food_stamp + with_social_security",
    data_pr, dropna=True,
)
results = model.fit(inference_method="blackjax_nuts",sample_kwargs={
                        "draws": 500,
                        "tune": 500,
                        "target_accept": 0.8}
                    ,cores=10, chains=10)



In [ ]:
# results = az.from_netcdf("data/processed/results_pr_model_21.nc")
az.plot_trace(results)

In [ ]:
results2 = az.from_netcdf("data/processed/processed/results_local_21.nc")
az.plot_trace(results2)

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

az.plot_posterior(results, var_names=["log_k_index"], ax=axes[0])
az.plot_posterior(results2, var_names=["log_k_index"], ax=axes[1])

axes[0].set_title("Empresas PR")
axes[1].set_title("Empresas Foreaneas")

plt.tight_layout()
plt.show()

In [ ]:
az.plot_trace(results)

In [ ]:
az.summary(results)

In [ ]:

az.plot_trace(results, var_names=["log_k_index"])

In [ ]:
az.summary(results)